In [ ]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append('../lib/*/*'); sys.path.append('..')
import lib.tools.Toolbox as tb
import lib.tools.TimeKeeper as tk
import metadata.trade_configs.Globals as gb
import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as pta
import lib.tools.Broker as br
import lib.tools.Scrivener as sc
import lib.tools.Logger as log
import glob
import os.path as p
from lib.tools.Config import Config
from lib.Portfolio import Portfolio
from lib.RoboTrader import RoboTrader
from lib.indicators.SqueezeMomentum import SqueezeMomentum
from lib.strategies.MomentumSeeking import MomentumSeeking
from lib.strategies.ParabolaSeeking import ParabolaSeeking
from lib.strategies.CombinedSeeking import CombinedSeeking
from lib.strategies.SlowSeeking import SlowSeeking

from lib.selectors.SimpleSelection import SimpleSelection
from lib.selectors.TuningSelection import TuningSelection

%matplotlib widget

In [ ]:
plt.close()
# symbols = ['NVDA','AMD','DELL', 'MSFT','META']#'TSM',
symbols = ['NVDA']
BANNED_DATES = [tk.d(5,27)]

def tuning(symbols, days, logging:bool = False, show:bool = False, show_success:bool = False, show_failure:bool = False, selector=SimpleSelection, strategy=MomentumSeeking, s:pd.Timestamp = None, e:pd.Timestamp = None, keep_history:bool = False):
    '''Runs a stock experiment on the given range of days. Days should be a negative number for how far back you want to look'''

    log.ENABLE_LOGGING = logging
    
    if isinstance(days, pd.Timestamp): dates = [days]
    elif isinstance(days, list): dates = days
    elif s and not e:
        dates = pd.date_range(start=s, end=tk.today(), freq='B')
        dates = dates[days:] if tk.is_after(gb.TRADE_START_HOUR, gb.TRADE_START_MINUTE, tk.now()) else dates[days-1:len(dates)-1]
    elif s and e:
        dates = pd.date_range(start=s, end=e, freq='B')
    else:
        dates = pd.date_range(start=tk.today()-pd.DateOffset(days=abs(days*3)), end=tk.today(), freq='B')
        dates = dates[days:] if tk.is_after(gb.TRADE_START_HOUR, gb.TRADE_START_MINUTE, tk.now()) else dates[days-1:len(dates)-1]
    conf, final_total = Config('base'), 0
    profit_tally = {key:0 for key in symbols}
    success, failure = [], []

    results = []
    print('Dates Covered\n',dates)
    previous_amount= 100000
    for date in dates:
        if date in BANNED_DATES: print(date,'banned by order of the Department of Normalcy'); continue
        trader = RoboTrader(None, conf, selector=selector, strategy=strategy, symbols=symbols)
        if keep_history: trader.port.cash = previous_amount
        for symbol in symbols: log.clear_logs(None, date, symbol)
        try: tickers = trader.trade_simulation(date, starting_capital=previous_amount)
        except FileNotFoundError: print('skipping',date); continue
        except TypeError: print('AHHH'); continue
        # except KeyError: print('AHHH'); continue
        # except ConnectionError: print('Alpaca is sleeping today')
        #tickers = trader.trade_simulation(date)
        day_profit = trader.port.cash-previous_amount
        if keep_history: previous_amount+=day_profit
        #if day_profit > 0: day_profit *= 1.8
        #else: day_profit *= .554
        
        if show:
            for ticker in tickers: tickers[ticker].plot_line()
            
        results.append({'date':date, 'profit': day_profit})

        print(str(date.month)+'/'+str(date.day), date.day_name(), round(day_profit,2),'\t\t',round(final_total,2))
        if len(trader.bulls) > 1:
            for symbol in trader.bulls:
                profit_tally[symbol] += tickers[symbol].ticker_profit
                print('\t'+symbol+':',trader.strategies[symbol].ticker_profit)

        final_total += day_profit
    if len(results) == 0: return
    results = pd.DataFrame(results); results.set_index('date', inplace=True)

    print('\nTOTAL:',round(results.profit.sum(),2), '\nDAILY:',
      round(results.profit.mean(),2), 'over',len(results),'days')
    if len(results.profit[results.profit < 0]) > 0:
        print('\tAverage Loss:',round(results.profit[results.profit < 0].mean(),2),
      '\n\tTotal Losses:',round(results.profit[results.profit < 0].sum(),2),
      '\n\tGreatest Loss in a Day',round(results.profit[results.profit < 0].min()))
    
    print('Profits by Symbol')
    for ticker in profit_tally: print('\t',ticker+':',profit_tally[ticker])
    return results


In [ ]:
def judge_effectiveness(df, count_vdiff:bool = True):
    sq = SqueezeMomentum(df)
    delta = df.close.diff()
    correct_tally, total = 0, 0
    for index in df.index:
        is_long = delta.loc[index] > 0 and (sq.indicator.loc[index].vdiff > 0) if count_vdiff else  delta.loc[index] > 0 and (sq.indicator.loc[index].value > 0)
        is_short = delta.loc[index] > 0 and (sq.indicator.loc[index].vdiff < 0) if count_vdiff else  delta.loc[index] < 0 and (sq.indicator.loc[index].value < 0)
        # is_long = delta.loc[index] > 0 and (sq.indicator.loc[index].vdiff > 0 or sq.indicator.loc[index].value > 0)
        # is_short = is_long = delta.loc[index] < 0 and (sq.indicator.loc[index].vdiff < 0 or sq.indicator.loc[index].value < 0)
        if is_long: correct_tally += 1
        elif is_short: correct_tally += 1
        total += 1
    return correct_tally/total

In [ ]:

# tuning(symbols, tk.d(4,10), False, True)
# tuning(symbols,
#  -1, False, True)metadata/trade_configs/s_MomentumSeeking_COIN.json
# tuning(symbols, tk.d(6,11), True, True)
# tuning(symbols, -1)
# tuning(symbols, tk.d(6,7), True, True)
# tuning(symbols, -1, True, True)
# results = tuning(symbols, -10, False, False, keep_history=True)
# tuning(symbols, -1, True, True)

# tuning(symbols, -10, False, False)
dates_of_comparison = [tk.d(6,6),tk.d(6,7),tk.d(6,17),tk.d(6,14)]
for date in dates_of_comparison:
    score = judge_effectiveness(sc.get_archive('NVDA',date))
    print(date, round(score,2))
# tuning(symbols, dates_of_comparison, True, True)

In [46]:
date = tk.d(6,18)
t_logs = log.get_log('NVDA', date)[tk.d(6,18,15,14):]
b_logs = log.get_log('NVDA', date, 'brokerage')[tk.d(6,18,15,14):]

for index in t_logs.index:
    if index not in b_logs.index:continue
    print("Checking ",index)
    try: 
        t_entry, b_entry = t_logs.loc[index], b_logs.loc[index]
        if isinstance(b_entry, pd.DataFrame): b_entry = b_entry.iloc[-1]
        if t_entry.volume != b_entry.volume: print(index, 'mismatch of volume,\n\tt_entry:',t_entry.volume,'\n\tb_entry:',b_entry.volume,'\n\tDiff:',t_entry.volume-b_entry.volume)
        if t_entry.close != b_entry.close: print(index, 'mismatch of close,\n\tt_entry:',t_entry.close,'\n\tb_entry:',b_entry.close,'\n\tDiff:',t_entry.close-b_entry.close)
        if t_entry.sq_dif != b_entry.sq_dif: print(index, 'mismatch of vdiff,\n\tt_entry:',t_entry.sq_dif,'\n\tb_entry:',b_entry.sq_dif,'\n\tDiff:',t_entry.sq_dif-b_entry.sq_dif)
    except KeyError:
        print('key error at',index)

Checking  2024-06-18 15:14:00-04:00
2024-06-18 15:14:00-04:00 mismatch of vdiff,
	t_entry: 0.0210000000000434 
	b_entry: 0.021000000000015 
	Diff: 2.8400892748692286e-14
Checking  2024-06-18 15:15:00-04:00
Checking  2024-06-18 15:16:00-04:00
2024-06-18 15:16:00-04:00 mismatch of vdiff,
	t_entry: 0.186649999999986 
	b_entry: 0.1866500000000144 
	Diff: -2.839395385478838e-14
Checking  2024-06-18 15:17:00-04:00
Checking  2024-06-18 15:18:00-04:00
2024-06-18 15:18:00-04:00 mismatch of vdiff,
	t_entry: 0.1185974999999928 
	b_entry: 0.1185974999999643 
	Diff: 2.8504976157250894e-14
Checking  2024-06-18 15:19:00-04:00
Checking  2024-06-18 15:20:00-04:00
Checking  2024-06-18 15:21:00-04:00
Checking  2024-06-18 15:22:00-04:00
Checking  2024-06-18 15:23:00-04:00
Checking  2024-06-18 15:24:00-04:00
Checking  2024-06-18 15:25:00-04:00
Checking  2024-06-18 15:26:00-04:00
Checking  2024-06-18 15:27:00-04:00
Checking  2024-06-18 15:28:00-04:00
2024-06-18 15:28:00-04:00 mismatch of volume,
	t_entry: 4

In [47]:
!python3 -m pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
api = br.paper_api()
api.get_clock().timestamp
t = api.get_clock().timestamp.second
while True:
    while t < gb.PIP_OFFSET:
        print('NOT QUITE THERE', t)
        tk.wait(0.001)
        t = api.get_clock().timestamp.second
    tk.sync()
    print('Our now:',tk.now())
    print('Their now:', api.get_clock().timestamp)
    t = api.get_clock().timestamp.second


In [27]:
us, them = tk.now(), api.get_clock().timestamp
print(us, them)
# if us > them:
#     diff = us - them
#     tk.wait(1-diff.total_seconds())
# else:
#     diff = them - us
#     tk.wait(1+diff.total_seconds())
tk.wait(1)
print('after our sync (should be one second later)')
us, them = tk.now(), api.get_clock().timestamp
print(us, them)


2024-06-18 14:53:25-04:00 2024-06-18 14:53:24.583788260-04:00
after our sync (should be one second later)
2024-06-18 14:53:26-04:00 2024-06-18 14:53:25.724170023-04:00


How do we test this? Simple: we implement and then test, if there's no difference going forwards, we've done it right.